In [ ]:
# # combine sessionmaker Session, with-statement & begin() to start session, handle exceptions, rollback, commit and close at once
# with Session.begin() as session:
#     session.add(object)
#     session.add_all([objects])
#     result = session.query(Project).filter(Project.name == 'Test').scalar() # scalar is good if only one match is possible due to database constraints

In [1]:
import settings # type: ignore
from camtrappy.db.testdb import initialize_new_project
from camtrappy.io import parsing

if not 'initialized' in locals():
    # Set data Dir & Data Project (file parser)
    DATA_DIR = settings.data_dir_animals
    new_project = parsing.ProjectParser(name='Test', projectfolder=None, datafolder=DATA_DIR, restrict_to=[])

    # Create new db project with parsed data
    initialize_new_project(new_project)
    initialized = True

home:      n:\Mitarbeiter\Florian\projects\CamTrapPy
data:      n:\Mitarbeiter\Florian\projects\CamTrapPy\tests\data
notebooks: n:\Mitarbeiter\Florian\projects\CamTrapPy\notebooks
--Setup complete--
Parsing folder: 7_Ittlingen
Parsing folder: 21_Herrenberg
Parsing folder: 25_Stockach
Parsing folder: best
Parsing folder: Location1
Parsing folder: Location2
2021-08-04 15:03:55,257 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-04 15:03:55,257 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("projects")
2021-08-04 15:03:55,257 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-04 15:03:55,267 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("projects")
2021-08-04 15:03:55,267 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-04 15:03:55,267 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("locations")
2021-08-04 15:03:55,267 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-08-04 15:03:55,267 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("locations")
2021-08-04 1

In [2]:
import settings # type: ignore
from camtrappy.db.testdb import Session
from camtrappy.db.schema import Video, Location, Object, VideoObject
from camtrappy.core.base import VideoList, VideoLoader

with Session.begin() as session:
    q = session.query(Video)
r = q.limit(10).all()
v = r[0]
print(v.fullpath)

with Session.begin() as session:
    q = session.query(Video).where(Video.location.has(name='7_Ittlingen')).order_by(Video.date, Video.time)
r = q.all()
print(r)

with Session.begin() as session:
    o = Object()
    vo = VideoObject(video_id=1, object=o, frames=[1,2,3], bboxes=[[0, 10, 50, 50], [10, 20, 50, 50]])
    vo2 = VideoObject(video_id=2, object=o)
    session.add_all([vo, vo2])
    
with Session.begin() as session:
    q = session.query(VideoObject)
r = q.first()
print(r.video_id, r.object.id, r.frames, r.bboxes)

2021-08-04 15:04:05,287 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-04 15:04:05,291 INFO sqlalchemy.engine.Engine SELECT videos.id AS videos_id, videos.path AS videos_path, videos.date AS videos_date, videos.time AS videos_time, videos.fps AS videos_fps, videos.duration AS videos_duration, videos.date_added AS videos_date_added, videos.location_id AS videos_location_id, (SELECT projects.datafolder 
FROM projects, locations 
WHERE projects.id = locations.project_id AND locations.id = videos.location_id) AS anon_1, (SELECT locations.folder 
FROM locations 
WHERE locations.id = videos.location_id) AS anon_2, (SELECT projects.datafolder 
FROM projects, locations 
WHERE projects.id = locations.project_id AND locations.id = videos.location_id) || ? || (SELECT locations.folder 
FROM locations 
WHERE locations.id = videos.location_id) || ? || videos.path AS anon_3 
FROM videos
 LIMIT ? OFFSET ?
2021-08-04 15:04:05,292 INFO sqlalchemy.engine.Engine [generated in 0.00087s] ('\\', '\\'

# Gutes Setting

gb7 = GaussianBlur((7, 7))

ce = CannyEdge(180, 400)

re = Resize()

bgs_mog = BgsMOG(250, 5, 0.7, 25, -1)

norm = Normalize()

bi = BilateralFilter(9)

base_transforms = [re]

compare_transforms = [bi, norm, bgs_mog, gb7, ce]  # ce is not so good for findContours

In [4]:
import settings # type: ignore
from camtrappy.db.testdb import Session
from camtrappy.core.base import VideoLoader, VideoPlayer
from camtrappy.core.transforms import (CannyEdge, GaussianBlur, TransformFactory,
                                       AdaptiveHistogram, Resize, Gamma, Hysteresis,
                                       BGS, Normalize, BilateralFilter, AdaptiveThreshold,
                                       BgsMOG, Threshold)

from camtrappy.core.analysis import CentroidTracker

at = AdaptiveThreshold(7, 3)
ah = AdaptiveHistogram(2, (12, 12))
gb3 = GaussianBlur((3, 3))
ce = CannyEdge(180, 400)
gamma = Gamma(0.25)
hyst = Hysteresis(0.15, 0.20)
bgs1 = BGS('KNN', 0.1)
bgs2 = BGS('KNN', 0.8)


bi = BilateralFilter(9) # smoothing that keeps the edges
norm = Normalize() # a little bit homogenization
bgs_mog = BgsMOG(hist=250, nmixtures=5, background_ratio=0.7,
                 noise=25, learningrate=-1) # background subtractor
gb7 = GaussianBlur((7, 7)) # some more smoothing
thr = Threshold(50, 'tozero') # eliminate some more noise


transforms = TransformFactory([bi, norm, bgs_mog, gb7, thr])

vl = VideoLoader(Session, 3, skip_n_frames=0) #, transforms=transforms)
object_tracker = CentroidTracker(min_area=30, eps=1.5, maxDisappeared=50, Session=Session)
VideoPlayer(vl).play(resize=Resize(), transforms=transforms, visitor=object_tracker, compare=True)

2021-09-24 07:52:01,837 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-24 07:52:01,838 INFO sqlalchemy.engine.Engine SELECT videos.id AS videos_id, videos.path AS videos_path, videos.date AS videos_date, videos.time AS videos_time, videos.fps AS videos_fps, videos.duration AS videos_duration, videos.date_added AS videos_date_added, videos.location_id AS videos_location_id, (SELECT projects.datafolder 
FROM projects, locations 
WHERE projects.id = locations.project_id AND locations.id = videos.location_id) AS anon_1, (SELECT locations.folder 
FROM locations 
WHERE locations.id = videos.location_id) AS anon_2, (SELECT projects.datafolder 
FROM projects, locations 
WHERE projects.id = locations.project_id AND locations.id = videos.location_id) || ? || (SELECT locations.folder 
FROM locations 
WHERE locations.id = videos.location_id) || ? || videos.path AS anon_3 
FROM videos 
WHERE videos.location_id = ? ORDER BY videos.date, videos.time
2021-09-24 07:52:01,839 INFO sqlalchemy.eng

Exception during reset or similar
Traceback (most recent call last):
  File "C:\Users\florian.frosch\AppData\Local\Continuum\anaconda3\envs\camtrappy-dev\lib\site-packages\sqlalchemy\pool\base.py", line 682, in _finalize_fairy
    fairy._reset(pool)
  File "C:\Users\florian.frosch\AppData\Local\Continuum\anaconda3\envs\camtrappy-dev\lib\site-packages\sqlalchemy\pool\base.py", line 887, in _reset
    pool._dialect.do_rollback(self)
  File "C:\Users\florian.frosch\AppData\Local\Continuum\anaconda3\envs\camtrappy-dev\lib\site-packages\sqlalchemy\engine\default.py", line 667, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 11128 and this is thread id 9784.
Exception closing connection <sqlite3.Connection object at 0x000000000C7D44E0>
Traceback (most recent call last):
  File "C:\Users\florian.frosch\AppData\Local\Continuum\anaconda3\envs\camtrappy-dev\lib\si

2021-09-24 07:52:20,112 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-09-24 07:52:20,114 INFO sqlalchemy.engine.Engine INSERT INTO objects DEFAULT VALUES
2021-09-24 07:52:20,114 INFO sqlalchemy.engine.Engine [cached since 132.4s ago] ()
2021-09-24 07:52:20,829 INFO sqlalchemy.engine.Engine INSERT INTO videos_objects (video_id, object_id, frames, bboxes, centroids, thumbnail) VALUES (?, ?, ?, ?, ?, ?)
2021-09-24 07:52:20,830 INFO sqlalchemy.engine.Engine [cached since 132.9s ago] (19, 59, '[2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0,  ... (875 characters truncated) ... 0, 204.0, 205.0, 206.0, 207.0, 208.0, 209.0, 210.0, 211.0, 212.0, 213.0, 214.0, 215.0, 216.0, 217.0, 218.0, 219.0, 220.0, 221.0, 222.0, 223.0, 224.0]', '[[152, 118, 13, 10], [149, 118, 18, 24], [146, 118, 20, 23], [144, 118, 18, 27], [141, 117, 21, 27], [138, 117, 20, 28], [135, 117, 20, 27], [133, 11 ... (3175 ch

In [16]:
import settings # type: ignore
from camtrappy.db.testdb import Session
from camtrappy.core.base import VideoLoader, VideoPlayer
from camtrappy.core.transforms import (CannyEdge, GaussianBlur, TransformFactory,
                                       AdaptiveHistogram, Resize, Gamma, Hysteresis,
                                       BGS, Normalize, BilateralFilter, AdaptiveThreshold,
                                       BgsMOG, BgsMOGMask, Threshold)

from camtrappy.core.analysis import CentroidTracker

bgs_Mask = BgsMOGMask(hist=250, nmixtures=5, background_ratio=0.7,
                      noise=25, learningrate=-1)
bi = BilateralFilter(9) # smoothing that keeps the edges
norm = Normalize() # a little bit homogenization
bgs_mog = BgsMOG(hist=250, nmixtures=5, background_ratio=0.7,
                 noise=25, learningrate=-1) # background subtractor
gb7 = GaussianBlur((7, 7)) # some more smoothing
thr = Threshold(200, 'tozero') # eliminate some more noise

ce = CannyEdge(180, 400)
gamma = Gamma(0.25)
hyst = Hysteresis(0.15, 0.20)


transforms = TransformFactory([gamma, bi, norm, bgs_Mask, thr])

vl = VideoLoader(Session, 4, skip_n_frames=9)
object_tracker = CentroidTracker(min_area=20, eps=1.5, maxDisappeared=25)
VideoPlayer(vl).play(resize=Resize(), transforms=transforms, visitor=object_tracker, compare=True)

2021-08-04 17:48:28,950 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-08-04 17:48:28,951 INFO sqlalchemy.engine.Engine SELECT videos.id AS videos_id, videos.path AS videos_path, videos.date AS videos_date, videos.time AS videos_time, videos.fps AS videos_fps, videos.duration AS videos_duration, videos.date_added AS videos_date_added, videos.location_id AS videos_location_id, (SELECT projects.datafolder 
FROM projects, locations 
WHERE projects.id = locations.project_id AND locations.id = videos.location_id) AS anon_1, (SELECT locations.folder 
FROM locations 
WHERE locations.id = videos.location_id) AS anon_2, (SELECT projects.datafolder 
FROM projects, locations 
WHERE projects.id = locations.project_id AND locations.id = videos.location_id) || ? || (SELECT locations.folder 
FROM locations 
WHERE locations.id = videos.location_id) || ? || videos.path AS anon_3 
FROM videos 
WHERE videos.location_id = ? ORDER BY videos.date, videos.time
2021-08-04 17:48:28,952 INFO sqlalchemy.eng